# Energy AI Hackathon 2024: Team Entelligence
Authors:
- Atharva Kalamkar, Computational Engineering
- Mihir Yogesh Bendre, Computational Engineering
- Avya Kalra, Computational Engineering
- Muhammad Taha, Computational Engineering
- Muhammad Akmal, PGE

**The University of Texas at Austin, Austin, Texas USA**

Executive Summary,
Workflow Goal,
Workflow Steps

# 1. Importing Libraries and Dataset

In [ ]:
#Importing required packages.
import pandas as pd
import numpy as np
import pickle
from sklearn import metrics
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
# To get diferent metric scores
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    confusion_matrix,
)

# 2. Data Overview

- Observations
- Sanity checks

# 3. Feature Selection / Parameter Importance

# 4. Data Preprocessing

* Dataset split into two: Training dataset & Prediction dataset
* 80% of them are used fro training the ML model, 20% are used for prediction to check ML performance
* Training data is split into 3 for K-fold cross-validation

# 5. Machine Leaning Models